In [16]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LinearRegression

In [17]:
# Read the sales data
file_path = Path("resources\machine_num.csv")
df_sales = pd.read_csv(file_path)
df_sales = df_sales.drop(columns=['date'])
# Display sample data
df_sales.head(3)

,setCode,price,Common,Rare,Uncommon,Mythical,Angle,Other,Human,Zombies,week_num
0,XLN,328,125,75,82,17,0,256,43,0,1
1,XLN,332,125,75,82,17,0,256,43,0,2
2,XLN,331,125,75,82,17,0,256,43,0,3


In [18]:
# Convert set code to binary with get dummies and concat to input df
num_df = pd.get_dummies(df_sales['setCode'])
frames = [num_df, df_sales]
result = pd.concat(frames, axis =1)
feed_df = result.drop(columns=['setCode'])
feed_df.head(3)

,KHM,MID,NEO,VOW,WAR,XLN,price,Common,Rare,Uncommon,Mythical,Angle,Other,Human,Zombies,week_num
0,False,False,False,False,False,True,328,125,75,82,17,0,256,43,0,1
1,False,False,False,False,False,True,332,125,75,82,17,0,256,43,0,2
2,False,False,False,False,False,True,331,125,75,82,17,0,256,43,0,3


In [4]:
# set machine targets and traininging vlues
y = feed_df['price']
X = feed_df.drop(columns=['price'])
print(X.shape, y.shape)

(127, 15) (127,)


In [5]:
# seperate test / train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

In [6]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [7]:
# Train the data
classifier.fit(X_train, y_train)

C:\Users\fumev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [8]:
# Score the model
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.29473684210526313
Testing Data Score: 0.0


In [9]:
# Predict outcomes for test data set and show the pecentage off 
predictions = classifier.predict(X_test)
df_percent = pd.DataFrame({"Prediction": predictions, "Actual": y_test})
df_percent['Percentage'] = (df_percent['Actual'] / df_percent['Prediction'] * 100 -100).round(2)
nonegs = []
for i in df_percent['Percentage']:
    if i > 0:
        nonegs.append(i)
    if i < 0:
        i = i * -1
        nonegs.append(i)
df_percent['Percentage2'] = nonegs
df_percent.head(3)

,Prediction,Actual,Percentage,Percentage2
121,225,218,-3.11,3.11
46,309,308,-0.32,0.32
73,287,277,-3.48,3.48


In [10]:
 # calculate the average percent error 
avg=  df_percent['Percentage2'].mean()
print(f'the average error is {avg}')

the average error is 4.643125


In [11]:
# Make Predictions for every column in  the sales df save to csv for graphing
predictions = classifier.predict(X)
price_prediction = df_sales
price_prediction["Predictions"] = predictions
price_prediction.to_csv('resources\prediction_m1.csv')
price_prediction.head(3)

,setCode,price,Common,Rare,Uncommon,Mythical,Angle,Other,Human,Zombies,week_num,Predictions
0,XLN,328,125,75,82,17,0,256,43,0,1,328
1,XLN,332,125,75,82,17,0,256,43,0,2,328
2,XLN,331,125,75,82,17,0,256,43,0,3,328


In [12]:
 ############Make Prediction for new data machine was givin first two weeks data time to predict its 25 weeks

In [13]:
#open new data to test
file_path = Path("resources\predict.csv")
new_data = pd.read_csv(file_path)
new_data = new_data.drop(columns=['setCode'])
new_data["KHM"] = 0
new_data["MID"] = 0
new_data["NEO"] = 0
new_data["VOW"] = 1
new_data["XLN"] = 0
new_data["WAR"] = 0
new_data = new_data[['KHM', 'MID', 'NEO', 'VOW', 'WAR', 'XLN','Common', 'Rare', 'Uncommon', 'Mythical', 'Angle', 'Other', 'Human', 'Zombies', 'week_num' ]]
new_data.head(3)

,KHM,MID,NEO,VOW,WAR,XLN,Common,Rare,Uncommon,Mythical,Angle,Other,Human,Zombies,week_num
0,0,0,0,1,0,0,142,168,137,58,1,389,84,31,1
1,0,0,0,1,0,0,142,168,137,58,1,389,84,31,2
2,0,0,0,1,0,0,142,168,137,58,1,389,84,31,3


In [14]:
# Make predictions with new data
new_prediction = classifier.predict(new_data)
new_data["Price Prediction"] = new_prediction
new_data.head(25)

,KHM,MID,NEO,VOW,WAR,XLN,Common,Rare,Uncommon,Mythical,Angle,Other,Human,Zombies,week_num,Price Prediction
0,0,0,0,1,0,0,142,168,137,58,1,389,84,31,1,316
1,0,0,0,1,0,0,142,168,137,58,1,389,84,31,2,316
2,0,0,0,1,0,0,142,168,137,58,1,389,84,31,3,316
3,0,0,0,1,0,0,142,168,137,58,1,389,84,31,4,316
4,0,0,0,1,0,0,142,168,137,58,1,389,84,31,5,316
5,0,0,0,1,0,0,142,168,137,58,1,389,84,31,6,316
6,0,0,0,1,0,0,142,168,137,58,1,389,84,31,7,316
7,0,0,0,1,0,0,142,168,137,58,1,389,84,31,8,316
8,0,0,0,1,0,0,142,168,137,58,1,389,84,31,9,316
9,0,0,0,1,0,0,142,168,137,58,1,389,84,31,10,316


In [15]:
#save to csv
new_data.to_csv('resources\m1VOW-pre.csv')